In [92]:
import sys
import os
from pathlib import Path

project_root = Path.cwd().parent 
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.predictive_models import (
    load_datasets,
    build_frequency_model,
    predict_frequency_model,
    build_sensationalism_model,
    predict_sensationalism_model,
    build_malicious_account_model,
    predict_malicious_account_model,
    build_naive_realism_model,
    predict_naive_realism_model,
)

import pandas as pd



# Ground Truth

In [93]:
ground_truth = pd.read_csv('../data/ground_truth.csv')
ground_truth

,statement,frequency_heuristic,malicious_account,sensationalism,naive_realism,link
0,A president steeped in constitutional lore mig...,2,0,2,2,https://www.cnn.com/2025/10/20/politics/trump-...
1,"Trump is 'committed' to $2,000 tariff dividend...",0,0,0,1,https://abc7.com/post/trump-is-committed-givin...
2,The longest US government shutdown in history ...,0,0,0,1,https://www.theguardian.com/us-news/2025/nov/1...
3,Add AP News as your preferred source to see mo...,0,0,0,0,https://apnews.com/article/redistricting-gerry...
4,"Key Points Prop 50 passed in California, appro...",1,0,1,0,https://azcapitoltimes.com/news/2025/11/12/pro...
5,"The bill will now head to the Senate, where it...",0,0,1,1,https://abcnews.go.com/Politics/house-vote-ful...
6,More than 6 in 10 registered voters said they ...,0,0,0,0,https://www.nbcnews.com/politics/politics-news...
7,"UK will not tolerate Chinese spying, minister ...",1,0,0,0,https://www.bbc.com/news/articles/c4gpnz05kr8o
8,Most Californians probably see the Capitol as ...,0,0,1,1,https://calmatters.org/commentary/2025/11/dana...
9,A new KFF/New York Times Survey of Immigrants ...,0,0,2,0,https://www.kff.org/racial-equity-and-health-p...


In [101]:
ground_truth.statement[11]

'Elon Musk, the world’s most successful businessman and President Donald Trump’s top White House adviser, said this week that the Department of Government Efficiency (DOGE) has uncovered shocking cases of fraud within the federal government.\n\nMusk made the remarks during a Monday meeting with the president and his cabinet secretaries.\n\n“One case of fraud was with the Small Business Administration, where they were handing out loans—$330 million worth of loans—to people under the age of 11,” Musk said. “I think the youngest, Kelly, was a 9-month-old who got a $100,000 loan.”\n\n“That’s a very precocious baby we’re talking about here,” Musk quipped.\n\nSBA Administrator Kelly Loeffler followed up Musk’s remarks by saying, “We’re tackling the fraud, waste, and abuse at the agency.”\n\n“We’ve seen, you know, hundreds of billions of dollars of fraud go unprosecuted, so we’re taking that on,” she said. “We have a zero-tolerance policy for fraud, and we continue to crack down on it and mak

In [102]:
print(ground_truth.link[11])

https://www.dailywire.com/news/musk-reveals-shocking-cases-of-fraud-found-at-small-business-administration?author=Daily+Wire+News&category=News&elementPosition=25&row=0&rowType=Vertical+List&title=Musk+Reveals+Shocking+Cases+Of+Fraud+Found+At+Small+Business+Administration


# Base Prompt

# Chain of Thought

In [73]:
cot_df = pd.read_csv('../webapp/cot_outputs.csv')
cot_df.iloc[16]

id                                 7a6dbf67-e02f-4e5c-ac56-f463bf8325d5
url                   https://www.mprnews.org/story/2025/11/18/minne...
freq_score                                                            0
freq_reason           The article reports on statements made by a po...
freq_confidence                                                     100
mal_score                                                             0
mal_reason            The article cites multiple, named, and verifia...
mal_confidence                                                      100
sens_score                                                            0
sens_reason           The tone of the article is neutral and informa...
sens_confidence                                                      95
naive_score                                                           0
naive_reason          The article presents a balanced and nuanced vi...
naive_confidence                                                

In [37]:
cot_df.head()

,id,url,freq_score,freq_reason,freq_confidence,mal_score,mal_reason,mal_confidence,sens_score,sens_reason,sens_confidence,naive_score,naive_reason,naive_confidence,overall_confidence
0,846eba04-9a07-4db2-8e2b-bd22b5e637ee,https://www.cnn.com/2025/10/20/politics/trump-...,2,The article heavily repeats the central narrat...,100,2,The article is a piece of speculative fiction ...,100,2,The article uses highly dramatic and exaggerat...,100,2,The article presents its critical perspective ...,100,100.00
1,fc5d28a2-d97d-4b17-97ad-897599641450,https://abc7.com/post/trump-is-committed-givin...,0,The article reports on a specific policy propo...,100,0,The sources cited are all credible and clearly...,100,1,While the article's own prose is neutral and o...,95,0,The article is a strong example of balanced re...,100,98.75
2,486dca50-6def-4ed3-b9eb-378771925237,https://www.theguardian.com/us-news/2025/nov/1...,0,The article reports on a singular event and it...,100,0,The article is a piece of standard political r...,100,1,While the headline is factual and the tone is ...,95,0,The article is well-balanced and explicitly pr...,100,98.75
3,adb23f07-5f8c-45a7-8bd6-8ebac3e55e91,https://apnews.com/article/redistricting-gerry...,0,The article introduces a central theme—mid-dec...,100,0,The article attributes information to credible...,100,1,The tone is largely informational and objectiv...,95,0,The article presents a nuanced situation by re...,90,96.25
4,c94d27e2-2d60-45ed-99f6-99edf4e4db0d,https://azcapitoltimes.com/news/2025/11/12/pro...,1,The article consistently repeats and reinforce...,95,0,The article is presented as a piece of politic...,100,0,The article maintains a neutral and analytical...,95,0,"The article is highly balanced and nuanced, ex...",100,97.50


# Function Calling

# Fractal Chain of Thought